In [1]:
import os
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [2]:
mapping_home = {'Richmond ':'RCH','Geelong ':'GEE','North Melbourne ':'NTH','Port Adelaide ':'POR','Gold Coast ':'GCS','Melbourne ':'MEL','GWS ':'GWS','Hawthorn ':'HAW','Carlton ':'CAR','Brisbane ':'BRL','Collingwood ':'COL','Adelaide ':'ADE','Fremantle ':'FRE','Western Bulldogs ':'WBD','Sydney ':'SYD','West Coast ':'WCE','St Kilda ':'STK','Essendon ':'ESS'}
mapping_away = {' Richmond':'RCH', ' Geelong':'GEE', ' North Melbourne':'NTH', ' Port Adelaide':'POR', ' Gold Coast':'GCS', ' Melbourne':'MEL', ' GWS':'GWS', ' Hawthorn':'HAW', ' Carlton':'CAR', ' Brisbane':'BRL', ' Collingwood':'COL', ' Adelaide':'ADE', ' Fremantle':'FRE', ' Western Bulldogs':'WBD', ' Sydney':'SYD', ' West Coast':'WCE', ' St Kilda':'STK', ' Essendon':'ESS',}

In [12]:
for year in range(1990, 2024):
    url = f'https://www.footywire.com/afl/footy/ft_match_list?year={year}'
    stat_tables = pd.read_html(url)
    tmp = stat_tables[9].drop(columns=[5,6]).reset_index(drop = True)
    if year == 2020:
        tmp = tmp[tmp[0].notna()].reset_index(drop = True)
    else:
        tmp = tmp.dropna().reset_index(drop = True)
    
    lst = list(tmp[tmp[0].str.contains('Round *')].index)

    r = 1
    output = pd.DataFrame({})
    for index in range(len(lst) - 1):
        try:
            tmp1 = tmp.iloc[lst[index]+1: lst[index + 1]]
            headers = list(tmp1.iloc[0])

            tmp1  = pd.DataFrame(tmp1.values[1:], columns=headers)
            tmp1['round'] = f'R{r}'

            r += 1

            output = pd.concat([output,tmp1])
        except:
            continue
    
    output['year'] = year
    output['Home'] = output['Home v Away Teams'].apply(lambda x: x.split('v')[0])
    output['Away'] = output['Home v Away Teams'].apply(lambda x: x.split('v')[1])
    output['Home Score'] = output['Result'].apply(lambda x: x.split('-')[0])
    output['Away Score'] = output['Result'].apply(lambda x: x.split('-')[1])
    output = output.drop(columns = ['Result','Home v Away Teams'])  
    
    for key in mapping_home.keys():
        output.loc[output['Home']==key, 'Home'] = mapping_home[key]
    for key in mapping_away.keys():
        output.loc[output['Away']==key, 'Away'] = mapping_away[key]
        
    output.to_csv(f'/Users/caseyh/Desktop/footy/roundresults/{year}_round_results.csv', index = False)